In [103]:
!pip install mlxtend

   ---------------------------------------- 0.0/1.4 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.4 MB ? eta -:--:--
   ------ --------------------------------- 0.2/1.4 MB 3.4 MB/s eta 0:00:01
   ---------------------- ----------------- 0.8/1.4 MB 7.3 MB/s eta 0:00:01
   ---------------------------------------- 1.4/1.4 MB 10.2 MB/s eta 0:00:00


In [104]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import mlxtend
from mlxtend.feature_selection import SequentialFeatureSelector

In [105]:
df = pd.read_csv("Carseats.csv")

In [106]:
df['Urban_Yes'] = pd.Series(np.zeros(df.shape[0]))
df.loc[df['Urban']=='Yes', 'Urban_Yes'] = 1

df['US_Yes'] = pd.Series(np.zeros(df.shape[0]))
df.loc[df['US']=='Yes', 'US_Yes'] = 1

df['ShelveLoc_rank'] = pd.Series(np.zeros(df.shape[0]))
df.loc[df['ShelveLoc']=='Good', 'ShelveLoc_rank'] = 2
df.loc[df['ShelveLoc']=='Medium', 'ShelveLoc_rank'] = 1

In [107]:
df

,Sales,CompPrice,Income,Advertising,Population,Price,ShelveLoc,Age,Education,Urban,US,Urban_Yes,US_Yes,ShelveLoc_rank
0,9.50,138,73,11,276,120,Bad,42,17,Yes,Yes,1.0,1.0,0.0
1,11.22,111,48,16,260,83,Good,65,10,Yes,Yes,1.0,1.0,2.0
2,10.06,113,35,10,269,80,Medium,59,12,Yes,Yes,1.0,1.0,1.0
3,7.40,117,100,4,466,97,Medium,55,14,Yes,Yes,1.0,1.0,1.0
4,4.15,141,64,3,340,128,Bad,38,13,Yes,No,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,12.57,138,108,17,203,128,Good,33,14,Yes,Yes,1.0,1.0,2.0
396,6.14,139,23,3,37,120,Medium,55,11,No,Yes,0.0,1.0,1.0
397,7.41,162,26,12,368,159,Medium,40,18,Yes,Yes,1.0,1.0,1.0
398,5.94,100,79,7,284,95,Bad,50,12,Yes,Yes,1.0,1.0,0.0


In [108]:
df['Sales'] = df['Sales'].astype('int64')
df['Urban_Yes'] = df['Urban_Yes'].astype('int64')
df['US_Yes'] = df['US_Yes'].astype('int64')
df['ShelveLoc_rank'] = df['ShelveLoc_rank'].astype('int64')

In [109]:
df.drop(['ShelveLoc', 'Urban', 'US'], axis=1, inplace=True)

In [110]:
df

,Sales,CompPrice,Income,Advertising,Population,Price,Age,Education,Urban_Yes,US_Yes,ShelveLoc_rank
0,9,138,73,11,276,120,42,17,1,1,0
1,11,111,48,16,260,83,65,10,1,1,2
2,10,113,35,10,269,80,59,12,1,1,1
3,7,117,100,4,466,97,55,14,1,1,1
4,4,141,64,3,340,128,38,13,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...
395,12,138,108,17,203,128,33,14,1,1,2
396,6,139,23,3,37,120,55,11,0,1,1
397,7,162,26,12,368,159,40,18,1,1,1
398,5,100,79,7,284,95,50,12,1,1,0


In [111]:
X = df.iloc[:, 1:11]
y = df.iloc[:, 0]

In [112]:
X.head()

,CompPrice,Income,Advertising,Population,Price,Age,Education,Urban_Yes,US_Yes,ShelveLoc_rank
0,138,73,11,276,120,42,17,1,1,0
1,111,48,16,260,83,65,10,1,1,2
2,113,35,10,269,80,59,12,1,1,1
3,117,100,4,466,97,55,14,1,1,1
4,141,64,3,340,128,38,13,1,0,0


In [113]:
y

0       9
1      11
2      10
3       7
4       4
       ..
395    12
396     6
397     7
398     5
399     9
Name: Sales, Length: 400, dtype: int64

In [114]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.3)

In [115]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
forward_feature_selection = SequentialFeatureSelector(RandomForestClassifier(n_jobs=-1),
                                                     k_features= (1,10),
                                                     forward=True,
                                                     floating = False,
                                                     verbose=2,
                                                     scoring= "accuracy",
                                                     cv= 9).fit(X_train,y_train)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
C:\Users\adam\anaconda3\Lib\site-packages\sklearn\model_selection\_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=9.
  warnings.warn(


In [ ]:
pd.DataFrame.from_dict(forward_feature_selection.get_metric_dict()).T

In [ ]:
import statsmodels.api as sm

In [ ]:
X = sm.add_constant(X_train)
model = sm.OLS(y_train, X)
model_fit = model.fit()
print(model_fit.summary())

In [ ]:
X = df[['ShelveLoc_rank','Age', 'Price', 'Income', 'Advertising', 'CompPrice']]
y = df['Sales']

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.3)

In [ ]:
X = sm.add_constant(X_train)
model = sm.OLS(y_train, X)
results = model.fit()
print(model_fit.summary())

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
lm = LinearRegression()

In [ ]:
lm.fit(X_train, y_train)

In [ ]:
predictions = lm.predict(X_test)

In [ ]:
sns.scatterplot(x = predictions, y=y_test)
plt.xlabel("Predictions")
plt.title("Sales vs Model Predictions")


In [ ]:
def predict(X, model):
    # the built-in get_prediction tool returns an array, so we need to convert to a dataframe
    predictions_df = pd.DataFrame(model.get_prediction(X).predicted, columns=['y_hat'], index=X.index)
    return predictions_df['y_hat']

def mse(y, y_hat):
    # calculate the residual error for each individual record
    resid = y - y_hat
    # square the residual (hence "squared error")
    sq_resid = resid**2
    # calculate the sum of squared errors
    SSR = sum(sq_resid)
    # divide by the number of records to get the mean squared error
    MSE = SSR / y.shape[0]
    return MSE

In [ ]:
def gains_chart(actuals, predictions, num_buckets=10):
    # Combine actuals and predictions into a single DataFrame
    results = np.asarray([actuals, predictions]).T
    # Sort by predictions descending
    results = results[results[:,1].argsort()[::-1]]

    # Calculate total number of instances and cumulative count
    total_count = len(results)
    cumulative_count = np.arange(1, total_count + 1) / total_count

    # Calculate cumulative actuals
    cumulative_actuals = np.cumsum(results[:,0])

    # Calculate cumulative baseline (if predictions were randomly sorted)
    baseline = np.arange(1, total_count + 1) * np.sum(actuals) / total_count

    # Calculate gains
    gains = cumulative_actuals / np.sum(actuals)

    # Calculate AUC (Area Under the Gains Curve)
    auc_score = auc(cumulative_count, gains)

    # Plotting the gains chart
    plt.figure(figsize=(10, 6))
    plt.plot(cumulative_count, gains, marker='o', linestyle='-', color='b', label=f'Gains Curve (AUC = {auc_score:.2f})')
    plt.plot([0, 1], [0, 1], linestyle='--', color='r', label='Baseline')
    plt.title('Gains Chart for Regression Model')
    plt.xlabel('Percentage of Population')
    plt.ylabel('Cumulative Gains')
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()

    # Print AUC
    print(f"Area Under the Gains Curve (AUC): {auc_score:.4f}")

def lift_chart_regression(actuals, predictions, num_buckets=10):
    # Create helper function for displaying lift charts
    # Combine actuals and predictions into a single DataFrame
    results = np.asarray([actuals, predictions]).T
    # Sort by predictions descending
    results = results[results[:,1].argsort()[::-1]]

    # Calculate total number of instances and cumulative count
    total_count = len(results)
    cumulative_count = np.arange(1, total_count + 1) / total_count

    # Calculate cumulative actuals
    cumulative_actuals = np.cumsum(results[:,0])

    # Calculate cumulative baseline (if predictions were randomly sorted)
    baseline = np.arange(1, total_count + 1) * np.sum(actuals) / total_count

    # Calculate lift
    lift = cumulative_actuals / baseline

    # Calculate cumulative lift
    cumulative_lift = np.cumsum(lift)

    # Plotting the lift chart
    plt.figure(figsize=(10, 6))
    plt.plot(cumulative_count, cumulative_lift, marker='o', linestyle='-', color='b', label='Lift Curve')
    plt.plot([0, 1], [1, 1], linestyle='--', color='r', label='Baseline')
    plt.title('Lift Chart for Regression Model')
    plt.xlabel('Percentage of Population')
    plt.ylabel('Lift')
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()

In [ ]:
predictions_train = predict(X_train[['ShelveLoc_rank','Price','Advertising','Age']], results)
print('mse train:',mse(y_train, predictions_train))
predictions_test = predict(X_test[['ShelveLoc_rank','Price','Advertising','Age']], results)
print('mse test:',mse(y_test, predictions_test))

In [ ]:
gains_chart(y_train,predictions_train)

In [ ]:
# Plot for ShelveLoc_rank
plt.figure(figsize=(10, 5))
plt.scatter(X_test['ShelveLoc_rank'], y_test, color='blue', label='Actual')
plt.scatter(X_test['ShelveLoc_rank'], predictions_test, color='red', label='Predicted')
plt.xlabel('ShelveLoc_rank')
plt.ylabel('Sales')
plt.title('Actual vs Predicted Sales (ShelveLoc_rank)')
plt.legend()
plt.grid(True)
plt.show()

# Plot for Price
plt.figure(figsize=(10, 5))
plt.scatter(X_test['Price'], y_test, color='blue', label='Actual')
plt.scatter(X_test['Price'], predictions_test, color='red', label='Predicted')
plt.xlabel('Price')
plt.ylabel('Sales')
plt.title('Actual vs Predicted Sales (Price)')
plt.legend()
plt.grid(True)
plt.show()

# Plot for Advertising
plt.figure(figsize=(10, 5))
plt.scatter(X_test['Advertising'], y_test, color='blue', label='Actual')
plt.scatter(X_test['Advertising'], predictions_test, color='red', label='Predicted')
plt.xlabel('Advertising')
plt.ylabel('Sales')
plt.title('Actual vs Predicted Sales (Advertising)')
plt.legend()
plt.grid(True)
plt.show()

# Plot for Age
plt.figure(figsize=(10, 5))
plt.scatter(X_test['Age'], y_test, color='blue', label='Actual')
plt.scatter(X_test['Age'], predictions_test, color='red', label='Predicted')
plt.xlabel('Age')
plt.ylabel('Sales')
plt.title('Actual vs Predicted Sales (Age)')
plt.legend()
plt.grid(True)
plt.show()
